# Weights and Biases

Goal: Train on CIFAR10 and do good by systematically finding better and better hyperparameters

## Set up groundwork

In [2]:
import torch as t
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from fancy_einsum import einsum
from typing import Union, Optional, Callable
import numpy as np
from einops import rearrange
from tqdm.notebook import tqdm_notebook
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import time
import wandb
import utilsD1 as utils

device = "cuda" if t.cuda.is_available() else "cpu"

In [3]:
# from tutorial
cifar_mean = [0.485, 0.456, 0.406]
cifar_std = [0.229,0.224,0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std)
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

utils.show_cifar_images(trainset, rows=3, cols=5)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
import resenet as myresnet

def train(trainset, testset, epochs: int, loss_fn: Callable, batch_size: int, lr: float) -> tuple[list, list]:

    model = myresnet.ResNet34().to(device).train()
    optimizer = t.optim.Adam(model.parameters(), lr=lr)

    loss_list = []
    accuracy_list = []

    trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
    testloader = DataLoader(testset, shuffle=True, batch_size=batch_size)

    for epoch in range(epochs):

        progress_bar = tqdm_notebook(trainloader)

        for (x, y) in progress_bar:

            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())

            progress_bar.set_description(f"Epoch = {epoch}, Loss = {loss.item():.4f}")

        with t.inference_mode():

            accuracy = 0
            total = 0

            for (x, y) in testloader:

                x = x.to(device)
                y = y.to(device)

                y_hat = model(x)
                y_predictions = y_hat.argmax(1)
                accuracy += (y_predictions == y).sum().item()
                total += y.size(0)

            accuracy_list.append(accuracy/total)

        print(f"Epoch {epoch+1}/{epochs}, train loss is {loss:.6f}, accuracy is {accuracy}/{total}")

    filename = "./w0d3_resnet.pt"
    print(f"Saving model to: {filename}")
    t.save(model.state_dict(), filename)

    utils.plot_results(loss_list, accuracy_list)
    return loss_list, accuracy_list

epochs = 1
loss_fn = nn.CrossEntropyLoss()
batch_size = 128
lr = 0.001

loss_list, accuracy_list = train(trainset, testset, epochs, loss_fn, batch_size, lr)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Sequential(
  (0): Conv2d(weights shape = torch.Size([10, 3, 3, 3]) stride = (1, 1) padding = (0, 0) )
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
)


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1/1, train loss is 1.181758, accuracy is 5939/10000
Saving model to: ./w0d3_resnet.pt


In [52]:
resnet = myresnet.ResNet34()
resnet.load_state_dict(t.load("w0d3_resnet.pt"))

<All keys matched successfully>

In [44]:
utils.show_cifar_images(trainset, rows=1, cols=1)

In [58]:
resnet(t.tensor(trainset[0][0]).unsqueeze(0))

/var/folders/38/0hzg6tlx5p3bm5hlwqmzl3mw0000gn/T/ipykernel_21932/451103362.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([[ 0.1739,  0.1367,  0.1418,  0.1431,  0.1319,  0.1091,  0.0877,  0.1173,
          0.1317,  0.1061, -0.1599, -0.0661, -0.1462, -0.0716, -0.1255, -0.0841,
         -0.0940, -0.1513, -0.1495, -0.1226, -0.1151, -0.0838, -0.0829, -0.0592,
         -0.0856, -0.1445, -0.1436, -0.1438, -0.0952, -0.1104, -0.1509, -0.1606,
         -0.0593, -0.1235, -0.0875, -0.1380, -0.0935, -0.1363, -0.1486, -0.0551,
         -0.0651, -0.0821, -0.1094, -0.1266, -0.1419, -0.1028, -0.1062, -0.0614,
         -0.0933, -0.0680, -0.1362, -0.0837, -0.1384, -0.0590, -0.1076, -0.0956,
         -0.1490, -0.1451, -0.0971, -0.1455, -0.1059, -0.0829, -0.0617, -0.1180,
         -0.0846, -0.0803, -0.0820, -0.1478, -0.0810, -0.1139, -0.1013, -0.1383,
         -0.1080, -0.1462, -0.0643, -0.1021, -0.0918, -0.0577, -0.0646, -0.0796,
         -0.1066, -0.1457, -0.0706, -0.1034, -0.1030, -0.1303, -0.0932, -0.1368,
         -0.1103, -0.1387, -0.1253, -0.0568, -0.1185, -0.0779, -0.1126, -0.0831,
         -0.0976, -0.0703, -